# Enrollment Days Calculator

Python script that takes into account the enrollment and school exit date to calculate the number of days a student was enrolled at the Local Education Agency (LEA).

In [1]:
# importing packages
import pandas as pd
import datetime as dt

In [2]:
# importing transfer data
student = pd.read_excel(r'/Users/scipio/Documents/Student_Transfers.xlsx',parse_dates = ['School Enrollment Date',
                                                                                         'School Unenrollment Date'])

In [3]:
# function to create SY 25 Enrollment Date
def sy_start (row):
    if row['School Enrollment Date'] < pd.to_datetime('2024-08-19'):
        return pd.to_datetime('2024-08-19')
    else:
        return row['School Enrollment Date']
    
student['SY 25 Enrollment Date'] = student.apply(sy_start, axis = 1)

# data cleaning
student['SY 25 Enrollment Date'] = student['SY 25 Enrollment Date'].dt.strftime('%Y-%m-%d')
student['School Enrollment Date'] = student['School Enrollment Date'].dt.strftime('%Y-%m-%d')
student['School Unenrollment Date'] = student['School Unenrollment Date'].dt.strftime('%Y-%m-%d')

In [4]:
student.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Student First Name        39 non-null     object
 1   Student Last Name         39 non-null     object
 2   Student School ID         39 non-null     int64 
 3   Student State ID          39 non-null     int64 
 4   Student Grade Level       39 non-null     object
 5   School Enrollment Date    39 non-null     object
 6   School Unenrollment Date  39 non-null     object
 7   SY 25 Enrollment Date     39 non-null     object
dtypes: int64(2), object(6)
memory usage: 2.6+ KB


In [5]:
# creating date range
start_date = pd.to_datetime('2024-08-19') #--> creating start date object
end_date = pd.to_datetime('2025-06-19')#--> creating end date object

# creating date range
date_range = pd.date_range(start = start_date, end = end_date)


# creating dataframe
df = pd.DataFrame({'date':date_range})

# creating DOW_num -- numeric representation of the DOW
df['dow_num'] = df['date'].dt.weekday

# creating month column
df['month'] = df['date'].dt.month

# creating year column 
df['year'] = df['date'].dt.year

# filtering out weekends 5-6
df = df[df['dow_num'] < 5]

# QA 
df['dow_num'].unique()

array([0, 1, 2, 3, 4])

In [6]:
# function to return DOW based on DOW_num column
def dow (row):
    if row['dow_num'] == 0:
        return 'Monday'
    elif row['dow_num'] == 1:
        return 'Tuesday'
    elif row['dow_num'] == 2:
        return 'Wednesday'
    elif row['dow_num'] == 3:
        return 'Thursday'
    elif row['dow_num'] == 4:
        return 'Friday'
    
df['dow'] = df.apply(dow, axis = 1)

In [7]:
# creating date ranges for breaks during the school year

#thanksgiving break date range
thanksgiving_break = pd.date_range(start = '2024-11-25', end = '2024-11-29')
thanksgiving_break = thanksgiving_break.to_list()

# christmas break date range
christmas_break = pd.date_range(start = '2024-12-23', end = '2025-01-03')
christmas_break = christmas_break.to_list()

# spring break date range
spring_break = pd.date_range(start = '2025-04-18', end = '2025-04-25')
spring_break = spring_break.to_list()

# days off list
days_off = ['2024-09-02', '2024-10-11','2024-10-14','2025-01-20','2025-02-14','2025-02-17', '2025-03-28','2025-05-26']


In [8]:
# function to account for unenrollment during school closing
def sy_end (row):
    if row['School Unenrollment Date'] == '2024-09-02':
        return '2024-08-30'
    elif row['School Unenrollment Date'] in ['2024-10-11','2024-10-14']:
        return '2024-10-10'
    elif row['School Unenrollment Date'] == '2025-01-20':
        return '2025-01-17'
    elif row['School Unenrollment Date'] in ['2025-02-14','2025-02-17']:
        return '2025-02-13'
    elif row['School Unenrollment Date'] == '2025-03-28':
        return '2025-03-27'
    elif row['School Unenrollment Date'] == '2025-05-26':
            '2025-05-23'
    elif row['School Unenrollment Date'] in thanksgiving_break:
        return '2025-11-22'
    elif row['School Unenrollment Date'] in christmas_break:
        return '2025-12-20'
    elif row['School Unenrollment Date'] in spring_break:
        return '2025-04-17'
    else:
        return row['School Unenrollment Date']

student['SY 25 Unenrollment Date'] = student.apply(sy_end, axis = 1)

In [9]:
# for loop to append thanksgiving break date ranges to days_off list
for d in thanksgiving_break:
    days_off.append(d.strftime('%Y-%m-%d'))

In [10]:
# for loop to append christmas break date ranges to days_off list
for d in christmas_break:
    days_off.append(d.strftime('%Y-%m-%d'))

In [11]:
# for loop to append spring break date ranges to days_off list
for d in spring_break:
    days_off.append(d.strftime('%Y-%m-%d'))

In [13]:
# total school days variable
total_school_days = 190

# returning in-service days for the school year
in_service = df[~df['date'].isin(days_off)] #--> returning dates not in the days_off list

# QA of school days and in
in_service.shape[0] == total_school_days

True

In [14]:
# reset index for count of school days
in_service = in_service.reset_index()

In [15]:
# creating empty list object
days_enrolled = []

# creating function to calculate the number of days a student was enrolled via iteration
def enrollment_days (enroll, exit): #--> enrollment parameters - start date and exit date
    """
    Function that takes into account the enrollment and exit date of a student
    
    2024-08-19 is the default enrollment date, if entered returns total school days
    
    If enrollment date doesn't != 2024-08-19 then total school days of exit date - total school days of enrollment date
    
    Appends output to enrollment_days list, if an error will return NA value.
    """
    try:
        if enroll == '2024-08-19':
            days_enrolled.append(in_service[in_service['date']== exit].index[0] + 1) #--> adding one to account for  0-index
        else:
            days_enrolled.append((in_service[in_service['date']== exit].index[0]) - (in_service[in_service['date']== enroll].index[0]) + 1)
    except:
            days_enrolled.append(pd.NA) #--> if error returns NA value


In [31]:
# creating function to calculate the number of days a student was enrolled via input
def enrollment_days_manual (enroll, exit): #--> enrollment parameters - start date and exit date
    """
    Function that takes into account the enrollment and exit date of a student
    
    2024-08-19 is the default enrollment date, if entered returns total school days
    
    If enrollment date doesn't != 2024-08-19 then total school days of exit date - total school days of enrollment date
    
    Appends output to enrollment_days list, if an error, will return error.
    """
    try:
        if enroll == '2024-08-19':
            return print('Enrollment Date:',enroll, '\nUnenrollment Date:',exit,
                         '\nEnrolled Days:',in_service[in_service['date']== exit].index[0] + 1) #--> adding one to account for  0-index
        else:
            return print('Enrollment Date:',enroll,'\nUnenrollment Date:',exit,
                         '\nEnrolled Days:',(in_service[in_service['date']== exit].index[0]) - (in_service[in_service['date']== enroll].index[0]) + 1)
    except:
            return 'Error' #--> if error returns NA value


In [32]:
# manual function testing
enrollment_days_manual('2024-08-20','2025-03-31')

Enrollment Date: 2024-08-20 
Unenrollment Date: 2025-03-31 
Enrolled Days: 138


In [16]:
# creating lists for function input
enroll_list = student['SY 25 Enrollment Date'].to_list()
exit_list = student['SY 25 Unenrollment Date'].to_list()

# for loop interating through the enroll_list and exit_list
for enroll,exit in zip(enroll_list,exit_list):
    enrollment_days(enroll,exit)
    
# assigning days_enrolled list to a column in a dataframe
student['Days Enrolled'] = days_enrolled

In [17]:
# displaying output
student[['Student School ID','SY 25 Enrollment Date','School Unenrollment Date','Days Enrolled']]

,Student School ID,SY 25 Enrollment Date,School Unenrollment Date,Days Enrolled
0,2392017,2024-08-19,2024-12-20,82
1,3192015,2024-08-19,2024-12-09,73
2,622507,2024-08-19,2024-12-05,71
3,202454,2024-08-19,2024-12-02,68
4,1972015,2024-08-19,2024-11-14,61
5,22227,2024-08-19,2024-10-22,44
6,36197345,2024-09-06,2024-10-22,31
7,4262014,2024-08-19,2024-10-21,43
8,2302016,2024-08-19,2024-10-21,43
9,4112023,2024-08-19,2025-03-03,120
